In [41]:
from jax import jit

from jwave import FourierSeries
from jwave.acoustics.time_varying import simulate_wave_propagation
from jwave.geometry import Domain, Medium, TimeAxis
from jwave.utils import load_image_to_numpy

import numpy as np
import matplotlib.pyplot as plt
import imageio
 
 
# Simulation parameters
N, dx = (256, 256), (0.1e-3, 0.1e-3)
domain = Domain(N, dx)
medium = Medium(domain=domain, sound_speed=1500.)
time_axis = TimeAxis.from_medium(medium, cfl=0.3, t_end=1.8e-05)

# Initial pressure field
p0 = load_image_to_numpy("gracias.png", image_size=N)/255.
p0 = FourierSeries(p0, domain)

# Compile and run the simulation
@jit
def solver(medium, p0):
  return simulate_wave_propagation(medium, time_axis, p0=p0)

pressure = solver(medium, p0)

In [44]:
for i in range(900):
    plt.imshow(pressure[i].params, cmap='seismic', vmin=-1, vmax=1)
    plt.axis('off')
    plt.savefig('gracias/frame_{:03d}.png'.format(i), bbox_inches='tight', pad_inches=0)
    plt.close()

# Crear una lista de frames a partir de las imágenes guardadas
frames = []
for i in range(900):
    filename = 'gracias/frame_{:03d}.png'.format(i)
    frames.append(imageio.imread(filename))

/tmp/ipykernel_91261/3858432538.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(filename))


In [45]:
imageio.mimsave('gracias.gif', frames, fps=30)